## In this notebook, we will explore how to do anomaly detection.

In [2]:
import pandas as pd

In [15]:
df = pd.read_csv("nyc_taxi.csv")
df.head()

,timestamp,value
0,2014-07-01 00:00:00,10844
1,2014-07-01 00:30:00,8127
2,2014-07-01 01:00:00,6210
3,2014-07-01 01:30:00,4656
4,2014-07-01 02:00:00,3820


In [16]:
def overview(df: pd.DataFrame, timestamp_col: str = None) -> None:
    print('Null Count:\n', df.isnull().sum(),'\n')
    print('Data Types:\n', df.dtypes)
    
    if timestamp_col is not None:
        df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors='coerce')

        print('\nDate Range:\n\nStart:\t',df[timestamp_col].min())
        print('End:\t',df[timestamp_col].max())
        print('Days:\t',(df[timestamp_col].max() - df[timestamp_col].min()))

In [17]:
overview(df, timestamp_col='timestamp')

Null Count:
 timestamp    0
value        0
dtype: int64 

Data Types:
 timestamp    object
value         int64
dtype: object

Date Range:

Start:	 2014-07-01 00:00:00
End:	 2015-01-31 23:30:00
Days:	 214 days 23:30:00


In [20]:
pip install holoviews bokeh

ERROR: Exception:
Traceback (most recent call last):
  File "/Users/ahmetcinci/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 107, in _run_wrapper
    status = _inner_run()
  File "/Users/ahmetcinci/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 98, in _inner_run
    return self.run(options, args)
  File "/Users/ahmetcinci/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/cli/req_command.py", line 71, in wrapper
    return func(self, options, args)
  File "/Users/ahmetcinci/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/commands/install.py", line 487, in run
    installed_versions[distribution.canonical_name] = distribution.version
  File "/Users/ahmetcinci/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/metadata/pkg_resources.py", line 189, in version
    return parse_version(self._dist.version)
  File "/Users/ahmetcinci/opt/anaconda3/lib/python3.9/site-packages/pip/_vendor/packaging/version

In [22]:
pip install --upgrade numpy

ERROR: Exception:
Traceback (most recent call last):
  File "/Users/ahmetcinci/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 107, in _run_wrapper
    status = _inner_run()
  File "/Users/ahmetcinci/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 98, in _inner_run
    return self.run(options, args)
  File "/Users/ahmetcinci/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/cli/req_command.py", line 71, in wrapper
    return func(self, options, args)
  File "/Users/ahmetcinci/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/commands/install.py", line 487, in run
    installed_versions[distribution.canonical_name] = distribution.version
  File "/Users/ahmetcinci/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/metadata/pkg_resources.py", line 189, in version
    return parse_version(self._dist.version)
  File "/Users/ahmetcinci/opt/anaconda3/lib/python3.9/site-packages/pip/_vendor/packaging/version

In [25]:
pip install --upgrade pip setuptools wheel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.2 MB/s  0:00:00 eta 0:00:01
  Attempting uninstall: wheel
    Found existing installation: wheel 0.37.0
    Uninstalling wheel-0.37.0:
      Successfully uninstalled wheel-0.37.0━━━━━ 0/2 [wheel]
  Attempting uninstall: setuptools━━━━━━━━━━ 0/2 [wheel]
    Found existing installation: setuptools 58.0.4━━━━━━━━━━━━━━━━ 1/2 [setuptools]
    Uninstalling setuptools-58.0.4:━━━━━━━━━━━━━━━━━━━ 1/2 [setuptools]
      Successfully uninstalled setuptools-58.0.4━━━━━━━━━━━━━━━━━━ 1/2 [setuptools]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [setuptools]2 [setuptools]
ERROR: Exception:
Traceback (most recent call last):
  File "/Users/ahmetcinci/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 107, in _run_wrapper
    status = _inner_run()
  File "/Users/ahmetcinci/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 98, in _inner_run
    return self.run(options, args)
  Fi

In [26]:
pip uninstall pyodbc
pip install pyodbc

SyntaxError: invalid syntax (3308578736.py, line 1)

In [27]:
pip check

anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.
applaunchservices 0.2.1 requires pyobjc, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
daal4py 2021.3.0 requires daal, which is not installed.
spyder 5.1.5 requires pyqt5, which is not installed.
spyder 5.1.5 requires pyqtwebengine, which is not installed.
cookiecutter 1.7.2 has requirement Jinja2<3.0.0, but you have jinja2 3.1.3.
cookiecutter 1.7.2 has requirement MarkupSafe<2.0.0, but you have markupsafe 2.1.5.
numba 0.54.1 has requirement numpy<1.21,>=1.17, but you have numpy 2.0.2.
pyarrow 15.0.0 has requirement numpy<2,>=1.16.6, but you have numpy 2.0.2.
scikit-learn 1.3.2 has requirement numpy<2.0,>=1.17.3, but you have numpy 2.0.2.
conda-verify 3.4.2 is not supported on this platform
Note: you may need to restart the kernel to use updated packages.


In [24]:
import numpy as np
print(np.__version__)

2.0.2


In [23]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

AttributeError: module 'numpy' has no attribute 'bool8'

In [18]:
Hourly = hv.Curve(df.set_index('timestamp').resample('H').mean()).opts(
    opts.Curve(title="New York City Taxi Demand Hourly", xlabel="", ylabel="Demand",
               width=700, height=300,tools=['hover'],show_grid=True))

Daily = hv.Curve(df.set_index('timestamp').resample('D').mean()).opts(
    opts.Curve(title="New York City Taxi Demand Daily", xlabel="", ylabel="Demand",
               width=700, height=300,tools=['hover'],show_grid=True))

Weekly = hv.Curve(df.set_index('timestamp').resample('W').mean()).opts(
    opts.Curve(title="New York City Taxi Demand Weekly", xlabel="Date", ylabel="Demand",
               width=700, height=300,tools=['hover'],show_grid=True))


(Hourly + Daily + Weekly).opts(shared_axes=False).cols(1)

NameError: name 'hv' is not defined